# Сбор данных с Web-scraping и API для социально-научных исследований
---
## Семинары 7-8. Понятие API. Работа с API ВКонтакте.

---
*ФСН, ОП "Политология", 2023-2024 гг.*

Лика Капустина,

lkapustina@hse.ru

**План занятия:**
1. [Генерируем данные для сетевого анализа](#par6)
---

**В этом документе мы с вами соберем и сгенерируем данные для того чтобы построить сеть своих друзей. Вам не нужно выполнять задачи и писать какой-то код кроме сохранения токена:**


**Основные ссылки**
- [Документация API ВКонтакте](https://dev.vk.com/ru/api/overview);
- [Как получить токен для работы с API ВКонтакте](https://github.com/lika1kapustina/hse_polit_web-scraping/blob/main/api_vk_get_token_and_test.ipynb).

Чтобы начать, давайте импортируем необходимые библиотеки

In [8]:
import requests      # запросы к серверу
import pandas as pd  # работа с таблицами
import numpy as np   # работа с арифметическими операциями
import json          # работа с файлами json

Введем сюда наши данные (хватит только токена) - как его сгенерировать смотрите в инструкции по ссылке выше.

**Запустите эту ячейку и сохраните в переменную ваш токен `token`**

In [ ]:
# запустите ячейку
token = input('Введите ваш токен тут: ') # вводим токен;
myid = token.split('user_id=')[1]        # получим ваш id пользователя из токена;
version = '5.199'                        # версия api вк, которую вы используете

## Генерируем данные для сетевого анализа<a name="par6"></a>

Ниже я предлагаю вам воспользоваться уже написанными мной функциями для сбора данных по друзьям и кратко поясню, что происходит и как вы можете сгенерировать сетевые данные с помощью API ВКонтакте:

In [4]:
from time import sleep # чтобы делать паузы
import tqdm # прогресс-бар

В ячейке ниже сохранены заранее написанные мной функции. Запустите эту ячейку:

In [13]:
def get_id_of_user_from_link(user_link, method='util.resolveScreeenName'):
    '''
    Функция get_id_of_user_from_link(user_link) принимает на вход ссылку на страницу пользователя ВКонтакте
    и возвращает его id.
    
    Args:
        :user_link: string - ссылка на пользователя;
    Returns:
        :user_id: string - id пользователя в виде строки
    Example:
        >>>> one_user_id = get_id_of_user_from_link('https://vk.com/lika.kapustina')
        >>>> one_user_id # 441721976
    '''

    screen_name = user_link.replace('https://vk.com/', '')

    # может быть такое, что ссылка на пользователя содержит id (vk.com/id777)
    if screen_name.startswith('id') == True and screen_name[2:].isdigit() == True:
        user_id = screen_name.replace('id', '')

    # если нет, нам нужно сделать запрос к ВК и получить id пользователя
    else:
        method_for_get_id = 'utils.resolveScreenName'
        parameters='screen_name=' + screen_name
        url = 'https://api.vk.com/method/' + method_for_get_id +'?'+ parameters + '&v=' + version + '&access_token=' + token
        response = requests.get(url)
        response = response.json()
        user_id = response['response']['object_id']
        
    # возвращаю id
    return user_id


def get_id_of_all_users_links(users_links, method='util.resolveScreeenName'):
    '''
    Функция get_ids_of_all_users_from_links(user_links) принимает на вход СПИСОК со ссылками на аккаунты пользователей,
    обращается внутри себя к методу `get_id_of_user_from_link(user_link)`,
    и возвращает СТРОКУ с id пользователей, записанных через строку.
    
    Args:
        :users_links: list - список с ссылками на пользователей;
    Returns:
        :users_ids_string: string - id пользователей, записанные через запятую в виде строки.
    Example:
        >>>> list_with_ids = get_id_of_all_users_links(['https://vk.com/lika.kapustina','https://vk.com/sdilov','https://vk.com/sofiagreseva'])
        >>>> list_with_ids
        
    '''
    users_ids_list = [] # создаем пустой список
    
    # идем по всему списку из ссылок
    for user_link in tqdm.tqdm(users_links):
        user_id = get_id_of_user_from_link(f'{user_link}') # получаем id пользователя
        users_ids_list.append(user_id)
        sleep(0.33) # небольшая пауза    
    
    users_ids_string = ','.join([str(i) for i in users_ids_list])
    
    return users_ids_string # возвращаем строку с id пользователей


def get_friends_of_user_only_ids(user_id, method='friends.get'):
    '''Функция get_friends_of_user_only_ids принимает на вход id пользователя ВКонтакте и возвращает строку, где через
    запятую перечислены id его друзей.
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :user_id: id пользователя ВКонтакте
    Returns:
        :friends_string: string с id друзей пользователя
    Example:
        >>>> limarenko_friends = get_friends_of_user('533710525')
        >>>> limarenko_friends # '626130,957188,1053362,2058801,3279648,4120747,5823842,8523846,11146055,11860274,25758390,169272036,171462727,182026369,196603818,322659228,462979979,520753933,538671248,550960397,550966564,551354669,551382456,551506228,578750417,606932606,616844912,633974128,660930698,665881497,670425420,689458339,703017057,703621379,708192084,708405503,710516910,710520153,710787650,711209497,711761122,711978083,711991565,712541914,712544833,712545388,713904031

    '''
    # осуществляем запрос
    parameters = 'count=5000' # берем максимум - 5000 друзей
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token + '&user_id=' + user_id
    response = requests.get(url)
    one_data = response.text
    
    # обрабатываем данные
    first_index = one_data.index('[')
    second_index = one_data.index(']')
    friends_string = one_data[first_index+1:second_index]
    return friends_string # возвращаем строку с id друзей пользователя


def get_friends_of_user_full_info_in_df(user_id, type_of = 'id', method='friends.get'):
    '''Функция get_friends_of_user_info_df принимает на вход ИЛИ id пользователя ИЛИ ССЫЛКУ ВКонтакте 
    и возвращает датафрейм, где перечислены данные его друзей.
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :user_id: id пользователя ВКонтакте;
        :type_of: тип введенных данных (id пользователя ('id')/ссылка на него('link')).
        
    Returns:
        :pandas.DataFrame: датафрейм с информацией о друзьях пользователя.
        
        > friend_owner: id пользователя, информация о чьих друзьях указана далее.
        > user_id: id пользователя;
        > user_name: имя пользователя;
        > user_surname: фамилия пользователя;
        > user_sex: пол пользователя;
        > user_domain: короткое имя пользователя (нужно для ссылки);
        > user_is_closed: закрыта ли страница пользователя;
        > user_link: сссылка на страницу пользователя.
        
    Example:
        >>>> limarenko_friends = get_friends_of_user('533710525')
        >>>> limarenko_friends # '626130,957188,1053362,2058801,3279648,4120747,5823842,8523846,11146055,11860274,25758390,169272036,171462727,182026369,196603818,322659228,462979979,520753933,538671248,550960397,550966564,551354669,551382456,551506228,578750417,606932606,616844912,633974128,660930698,665881497,670425420,689458339,703017057,703621379,708192084,708405503,710516910,710520153,710787650,711209497,711761122,711978083,711991565,712541914,712544833,712545388,713904031

    '''
    
    # если type_of = 'link' (то есть, не id), нужно получить id пользователя
    if type_of != 'id':
        link = user_id #значит, нам попалась ссылка и из нее нужно вынуть screen_name пользователя
        screen_name = link.replace('https://vk.com/', '')
        
        # может быть такое, что ссылка на полььзователя содержит id (vk.com/id777)
        if screen_name.startswith('id') == True and screen_name[2:].isdigit() == True:
            user_id = screen_name.replace('id', '')
        
        # если нет, нам нужно сделать запрос к ВК и получить id пользователя
        else:
            method_for_get_id = 'utils.resolveScreenName'
            parameters='screen_name=' + screen_name
            url = 'https://api.vk.com/method/' + method_for_get_id +'?'+ parameters + '&v=' + version + '&access_token=' + token
            response = requests.get(url)
            response = response.json()
            user_id = response['response']['object_id']

    
    # осуществляем запрос
    parameters = 'count=5000' + '&extended=1' + '&fields=bdate,city,country,sex,domain' + '&name_case=nom' # берем максимум - 5000 друзей
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token + '&user_id=' + str(user_id)
    response = requests.get(url)
    response = response.json()
    
    # собираем данные для датафрейма
    number_of_friends = response['response']['count'] # число друзей у пользователя
    
    # список: чьи друзья в этом датафрейме?
    owner_of_friends = [user_id] * number_of_friends
    
    
    # общая информация, получается без необходимости использовать try-except.
    friends_ids = [response['response']['items'][i]['id'] for i in range(number_of_friends)]
    friends_first_names = [response['response']['items'][i]['first_name'] for i in range(number_of_friends)]
    friends_second_names = [response['response']['items'][i]['last_name'] for i in range(number_of_friends)]
    friends_sexs = [response['response']['items'][i]['sex'] for i in range(number_of_friends)]
    friends_is_closed = [response['response']['items'][i]['is_closed'] for i in range(number_of_friends)]


    
    # информация, которую нужно получать через try-except
    friends_domains = []
    for i in range(number_of_friends):
        try:
            one_domain = response['response']['items'][i]['domain']
        except:
            one_domain = str(response['response']['items'][i]['id'])
        friends_domains.append(one_domain)
        
        
    ## ссылки на пользвателей; если есть domain - значит, с domain; если нет - значит, с id
    friends_links = list(map(lambda x: f'https://vk.com/{x}' if x.isdigit() == False else \
                       f'https://vk.com/id{x}', friends_domains))
        
    # собираем датафрейм
    friends_df = pd.DataFrame({'friend_owner': owner_of_friends,
                               'user_id': friends_ids,
                              'user_name': friends_first_names,
                              'user_surname': friends_second_names,
                              'user_sex': friends_sexs,
                              'user_domain': friends_domains,
                              'user_is_closed': friends_is_closed,
                              'user_link': friends_links})
    return friends_df


def get_friends_of_a_few_users_full_info_in_df(users_ids, type_of='id', method='friends.get'):
    '''Функция get_friends_of_a_few_users_info_df принимает на вход строку с id пользователей ВКонтакте,
    записанными через запятую, обращается к функции get_friends_of_user_full_info_in_df(user_id),
    и возвращает pandas.DataFrame с информацией о всех друзьях всех указанных пользователей
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :users_ids: string: ИЛИ id пользователей ВКонтакте ИЛИ ссылки на страницы пользователей ВКонтакте, записанных через запятую;
        :type_of: string: тип данных на входе (по умолчанию 'id' - id пользователей), либо 'link' - ссылки на пользователей.
    
    Returns:
        :pandas.DataFrame: датафрейм с информацией о друзьях пользователей, чьи id переданы функции.
        > friend_owner: id пользователя, информация о чьих друзьях указана далее.
        > user_id: id пользователя;
        > user_name: имя пользователя;
        > user_surname: фамилия пользователя;
        > user_sex: пол пользователя;
        > user_domain: короткое имя пользователя (нужно для ссылки);
        > user_is_closed: закрыта ли страница пользователя;
        > user_link: сссылка на страницу пользователя.
        
    Example:
        >>>> some_friends = get_friends_of_a_few_users_full_info_in_df('48905537,12900237')
        >>>> some_friends
        
    Exmple:
        >>>> my_friends_list = ['https://vk.com/zzimablue',
                                   'https://vk.com/ksperov',
                                   'https://vk.com/sofiagreseva',
                                   'https://vk.com/sdilov',
                                   'https://vk.com/narepapoyan',
                                   'https://vk.com/kirillmuzyka']
        >>>> my_friends_string = ','.join(my_friends_list)
        >>>> friends_of_my_friends = get_friends_of_a_few_users_full_info_in_df(my_friends_string, type_of='link')
        >>>> friends_of_my_friends
    '''
    
    users_ids = users_ids.split(',')
    
    full_users_friends_data = pd.DataFrame()
    
    for user_id in tqdm.tqdm(users_ids):
        try:
            one_user_friends_data = get_friends_of_user_full_info_in_df(user_id,
                                                                       type_of = type_of)
            full_users_friends_data = pd.concat([full_users_friends_data, one_user_friends_data])
            sleep(0.33)
        except Exception as e:
            print(e)
    
    return full_users_friends_data

Теперь запустим код ниже:

**Сперва введите сюда ссылку на ваш профиль:**

In [6]:
# вводим ссылку на ваш профиль:
vk_profile_link = input('Введите ссылку на ваш профиль ВКонтакте: ') # моя, например, 'https://vk.com/lika.kapustina'

Введите ссылку на ваш профиль ВКонтакте: https://vk.com/lika.kapustina


В этой ячейке происходит генерация датафрейма `friends_of_my_friends` - это **датафрейм со всеми друзьями ваших друзей**. 

**Пара заметок:**
* Если вы столкнетесь с ошибкой ```HTTPSConnectionPool(host='api.vk.com', port=443)``` или ошибкой ```('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))```, это связано с нестабильностью сети. Код выше написан с использованием конструкции `try-except`, поэтому столкновение с ошибкой сети не вызовет проблем.
* Возможно, код выше не оптимален с точки зрения организации запросов, но вы при желании можете его организовать;
* Скорость сбора даных зависит от вашего числа друзей, потому что в ходе выполнения функции мы отправяем запрос на получение списка друзей всех ваших друзей.

In [ ]:
friends_data = get_friends_of_user_full_info_in_df(vk_profile_link, type_of = 'link') # получаем список друзей

my_friends_list = friends_data['user_link'].tolist() # получаем список друзей ваших друзей
my_friends_string = ','.join(my_friends_list)        # склеиваем idшники друзей в строку, разделяемым запятыми
 
# а теперь пользуемся функцией get_friends_of_a_few_users_full_info_in_df - 
# и получаем датафрейм с вашими друзьями 
# и друзьями всех друзей ваших друзей (людей, которых я знаю лично и через 1 рукопожатие)
friends_of_my_friends = get_friends_of_a_few_users_full_info_in_df(my_friends_string, type_of='link')

Что в этом датафрейме? Датафрейм содержит информацию о друзьях всех моих друзей.

* Колонка `friend_owner` – это id пользователя, информация о чьем друге отражается в строке;
* Колонка `user_id` – это id пользователя, информация о октором отражает в строке. Это один из друзей пользователя с id `friend_owner`.

В этом примере: `Елена Карпова` с id `user_id` является другом пользователя с id (в `friend_owner`) `130896`, и т.д. **То есть - одна строчка - это информация не об одном пользователе, а об одной дружеской связи!**

In [175]:
friends_of_my_friends # датафрейм содержит 325 тысяч строк - в моем примере

,friend_owner,user_id,user_name,user_surname,user_sex,user_domain,user_is_closed,user_link
0,130896,1450,Елена,Карпова,1,karmannoe_zlo,False,https://vk.com/karmannoe_zlo
1,130896,6487,Михаил,Шлемин,2,id6487,False,https://vk.com/id6487
2,130896,9337,Даниил,Олегович,2,hashmaker,False,https://vk.com/hashmaker
3,130896,11446,Константин,Печенежский,2,konstantin_p,False,https://vk.com/konstantin_p
4,130896,12662,Григорий,Григорьев,2,evil_gr,False,https://vk.com/evil_gr
...,...,...,...,...,...,...,...,...
41,821924911,601140432,Михаил,Иванов,2,mibog108,False,https://vk.com/mibog108
42,821924911,612597498,Дмитрий,Котов,2,id612597498,False,https://vk.com/id612597498
43,821924911,720390827,Юрий,Слепичев,2,yurimj,False,https://vk.com/yurimj
44,821924911,736387180,Michel,Angelo,2,imshvets,False,https://vk.com/imshvets


**Обработаем датафрейм: пусть в нем останется только информация о тех дружеских связях, в которых оба пользователя являются вашими друзьями**

In [176]:
my_friends_ids = friends_data['user_id'].tolist() # сохраняем список только моих друзей в список
my_friends_ids.append(friends_data['friend_owner'].unique()[0]) # добавляем в список себя

# создаем small_df, сбрасываем дупликаты
small_df = friends_of_my_friends[['user_id', 'user_name', 'user_surname']].drop_duplicates()
# мерджим датафреймы
friends_of_my_friends = pd.merge(friends_of_my_friends, small_df, left_on='friend_owner', right_on='user_id')
friends_of_my_friends = friends_of_my_friends[['user_id_x', 'user_name_x', 'user_surname_x', 'user_id_y', 'user_name_y', 'user_surname_y']]

# Создаем колонки с полными именами пользователей
friends_of_my_friends['user_x'] = friends_of_my_friends['user_name_x'] + ' ' + friends_of_my_friends['user_surname_x']
friends_of_my_friends['user_y'] = friends_of_my_friends['user_name_y'] + ' ' + friends_of_my_friends['user_surname_y']

# Оставляем в friends_of_my_friends только тех пользователей, которые есть у вас в друзьях:
friends_of_my_friends = friends_of_my_friends[(friends_of_my_friends['user_id_x'].isin(my_friends_ids)) &\
                     friends_of_my_friends['user_id_y'].isin(my_friends_ids)]
friends_of_my_friends # финальный датафрейм!

,user_id_x,user_name_x,user_surname_x,user_id_y,user_name_y,user_surname_y,user_x,user_y
784,12325849,Андрей,Гречко,130896,Егор,Юрескул,Андрей Гречко,Егор Юрескул
963,38031682,Юлиан,Баландин,130896,Егор,Юрескул,Юлиан Баландин,Егор Юрескул
968,38584695,Федор,Духновский,130896,Егор,Юрескул,Федор Духновский,Егор Юрескул
987,42187308,Иван,Александров,130896,Егор,Юрескул,Иван Александров,Егор Юрескул
994,43907854,Мария,Учаева,130896,Егор,Юрескул,Мария Учаева,Егор Юрескул
...,...,...,...,...,...,...,...,...
296989,202296311,Данила,Морозов,821924911,Александр,Матвиенко,Данила Морозов,Александр Матвиенко
296997,241798539,Николай,Лобов,821924911,Александр,Матвиенко,Николай Лобов,Александр Матвиенко
297002,274066679,Дарья,Рахмалёва,821924911,Александр,Матвиенко,Дарья Рахмалёва,Александр Матвиенко
297013,441721976,Lika,Kapustina,821924911,Александр,Матвиенко,Lika Kapustina,Александр Матвиенко


Как интерпретировать данные выше? Этот тип данных дальше можно будет использования для создания сетей. В колонке `user_x` находится пользователь, от которого идет ребро; в колонке `user_y` находится пользователь, к которому идет ребро. Переводя на человеческий язык:

* `Юлиан Баландин` - `user_x` является другом `Егора Юрескула` - `user_y`;
* `Федор Духновский` - `user_x` является другом `Егора Юрескула` - `user_y`;
* `Иван Александров` - `user_x` является другом `Егора Юрескула` - `user_y`, и т. д.

Давайте сохраним наши данные в .xlsx формате:

In [180]:
friends_of_my_friends.to_excel('my_friends_network_31_01_2024.xlsx')

А теперь прочитаем их заново и создадим объект `GEXF` для дальнейшей работы в GEFI. Для этого импортируем пакет `networkx`, позволяющий строить сети и работать с сетевыми объектами в Python.

<font color='orange'>Если на моменте импорта `networkx` вы столкнулись с ошибкой</font> ```ImportError: cannot import name 'gcd' from 'fractions' (/opt/anaconda3/lib/python3.9/fractions.py)```, откройте этот ноутбук в **Google Colab** и продолжите работу там.

In [184]:
import networkx as nx # импортируем пакет networkx как nx для того чтобы работать с сетями

In [ ]:
df = pd.read_excel('my_friends_network_31_01_2024.xlsx') # сохраняем в df наш датафрейм

Тут мы используем функцию `.from_pandas_edgelist()`, которая позволяет создать сеть на основе датафрейма `pandas`, где `user_x` - это пользователь, от которого исходит ребро, а `user_y` - пользователь, к которому идет ребро. Запустим код ниже и сохраним нашу сеть в переменную `G`.

In [ ]:
G = nx.from_pandas_edgelist(df, 'user_x', 'user_y')

In [ ]:
G # сетевой объект

А теперь сохраним эту самую сеть в формате `.gexf`, чтобы дальше мы могли открыть этот файл в Gephi.

In [ ]:
nx.write_gexf(G, "my_friends_network_31_01_2024.gexf") # сохраняем

А теперь давайте откроем **Gephi Lite** – недавно вышедшую онлайн-версию программы Gephi, которую вы можете открыть онлайн.

* Откройте ссылку: https://gephi.org/gephi-lite/ ;
* Выберите `open local graph`, и откройте документ `my_friends_network_31_01_2024.gexf`;
* Откройте `Statistics` и посчитайте `degree` и `Louvian community detection` метрики;
* Откройте `Appearence (node)` и установите цвет в зависимости от модулярности;
* Откройте `Layout` и поэкспериментируйте с позицией сети.

**Обсуждали ли вы модулярность и метод выявления сообществ Лувена?**

## Заключение<a name="parlast"></a>

Сегодня мы с вами обсудили API ВКонтакте. Надеюсь, это было полезно для вас. То, что я советую вам помнить:
* **Если можно использовать API - используйте API**;
* **Запомните способ, с помощью которого генерируются запросы к API ВКонтакте**;
* **Работаешь с API и не понимаешь что делать? Обращайся к документации!**